In [1]:
# SERVER EDITS TODO:
# - uncomment out pip install line
# - set dummy_submission to is_submit
# - delete THIS


import torch as tc 
import torch.nn as nn  
import numpy as np
from tqdm import tqdm
import os,sys,cv2
from torch.cuda.amp import autocast
import matplotlib.pyplot as plt
import albumentations as A
# !python -m pip install --no-index --find-links=/kaggle/input/pip-download-for-segmentation-models-pytorch segmentation-models-pytorch
import segmentation_models_pytorch as smp
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DataParallel
from glob import glob
import tifffile as tiff
import pandas as pd
from dotenv import load_dotenv

/tmp/ipykernel_47377/3210523468.py:22: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
p_augm = 0.05 #0.5
#add rotate.  less p_augm

class CFG:
    # ============== pred target =============
    target_size = 1

    # ============== model CFG =============
    model_name = 'Unet'
    router_backbone = 'resnext50_32x4d'
    backbones = [
        'resnext50_32x4d',
        'resnext50_32x4d',
        'resnext50_32x4d',
        'resnext50_32x4d',
        # 'resnext101_32x8d',
        # 'se_resnext50_32x4d',
        # 'mit_b2'
    ]

    in_chans = 1   #5 # 65
    # ============== training CFG =============
    image_size = 1024 # 512 # 512
    input_size = 1024 # 512 # 512

    train_batch_size = 2 #4 #16
    valid_batch_size = 2

    epochs = 31 #30 #25
    lr = 8e-4
    chopping_percentile=1e-5
    # ============== fold =============
    valid_id = 1


    # ============== augmentation =============
    train_aug_list = [
        A.Rotate(limit=270, p= 0.5),
        A.RandomScale(scale_limit=(0.8,1.25),interpolation=cv2.INTER_CUBIC,p=p_augm),
        A.RandomCrop(input_size, input_size,p=1),
        A.RandomGamma(p=p_augm*2/3),
        A.RandomBrightnessContrast(p=p_augm,),
        A.GaussianBlur(p=p_augm),
        A.MotionBlur(p=p_augm),
        A.GridDistortion(num_steps=5, distort_limit=0.3, p=p_augm),
        ToTensorV2(transpose_mask=True),
    ]
    train_aug = A.Compose(train_aug_list)
    valid_aug_list = [
        ToTensorV2(transpose_mask=True),
    ]
    valid_aug = A.Compose(valid_aug_list)

In [3]:
def to_1024(img , image_size = 1024):
    if image_size > img.shape[1]:
       img = np.rot90(img)
       start1 = (CFG.image_size - img.shape[0])//2 
       top =     img[0                    : start1,   0: img.shape[1] ]
       bottom  = img[img.shape[0] -start1 : img.shape[0],   0 : img.shape[1] ]
       img_result = np.concatenate((top,img,bottom ),axis=0)
       img_result = np.rot90(img_result)
       img_result = np.rot90(img_result)
       img_result = np.rot90(img_result)
    else :
       img_result = img
    return img_result

def to_1024_no_rot(img, image_size = 1024):
    if image_size > img.shape[0]:  
       start1 = ( image_size - img.shape[0])//2
       top =     img[0                    : start1,   0: img.shape[1] ]
       bottom  = img[img.shape[0] -start1 : img.shape[0],   0 : img.shape[1] ]
       img_result = np.concatenate((top,img,bottom ),axis=0)
    else: 
       img_result = img
    return img_result

#  add border
def to_1024_1024(img  , image_size = 1024 ):
     img_result = to_1024(img, image_size )
     return img_result
    
#  drop border
def to_original ( im_after, img, image_size = 1024 ):
    top_ = 0
    left_ = 0
    if (im_after.shape[0] > img.shape[0]):
             top_  = ( image_size - img.shape[0])//2 
    if    (im_after.shape[1] > img.shape[1]) :
             left_  = ( image_size - img.shape[1])//2  
    if (top_>0)or (left_>0) :
             img_result = im_after[top_                    : img.shape[0] + top_,   left_: img.shape[1] + left_ ]
             #print(im_after.shape,'-->',img_result.shape)
    else:
             img_result = im_after
    return img_result  

In [4]:
def min_max_normalization(x:tc.Tensor)->tc.Tensor:
    """input.shape=(batch,f1,...)"""
    shape=x.shape
    if x.ndim>2:
        x=x.reshape(x.shape[0],-1)
    
    min_=x.min(dim=-1,keepdim=True)[0]
    max_=x.max(dim=-1,keepdim=True)[0]
    if min_.mean()==0 and max_.mean()==1:
        return x.reshape(shape)
    
    x=(x-min_)/(max_-min_+1e-9)
    return x.reshape(shape)

def norm_with_clip(x:tc.Tensor,smooth=1e-5):
    dim=list(range(1,x.ndim))
    mean=x.mean(dim=dim,keepdim=True)
    std=x.std(dim=dim,keepdim=True)
    x=(x-mean)/(std+smooth)
    x[x>5]=(x[x>5]-5)*1e-3 +5
    x[x<-3]=(x[x<-3]+3)*1e-3-3
    return x

def add_noise(x:tc.Tensor,max_randn_rate=0.1,randn_rate=None,x_already_normed=False):
    """input.shape=(batch,f1,f2,...) output's var will be normalizate  """
    ndim=x.ndim-1
    if x_already_normed:
        x_std=tc.ones([x.shape[0]]+[1]*ndim,device=x.device,dtype=x.dtype)
        x_mean=tc.zeros([x.shape[0]]+[1]*ndim,device=x.device,dtype=x.dtype)
    else: 
        dim=list(range(1,x.ndim))
        x_std=x.std(dim=dim,keepdim=True)
        x_mean=x.mean(dim=dim,keepdim=True)
    if randn_rate is None:
        randn_rate=max_randn_rate*np.random.rand()*tc.rand(x_mean.shape,device=x.device,dtype=x.dtype)
    cache=(x_std**2+(x_std*randn_rate)**2)**0.5
    #https://blog.csdn.net/chaosir1991/article/details/106960408
    
    return (x-x_mean+tc.randn(size=x.shape,device=x.device,dtype=x.dtype)*randn_rate*x_std)/(cache+1e-7)
 
class Data_loader(Dataset):
     
    def __init__(self,paths,is_label):
        self.paths=paths
        self.paths.sort()
        self.is_label=is_label
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,index):
         
        img = cv2.imread(self.paths[index],cv2.IMREAD_GRAYSCALE)
        
        img = to_1024_1024(img , image_size = CFG.image_size ) #  to_original( im_after, img_save, image_size = 1024)

        img = tc.from_numpy(img.copy())
        if self.is_label:
            img=(img!=0).to(tc.uint8)*255
        else:
            img=img.to(tc.uint8)
        return img

def load_data(paths,is_label=False):
    data_loader=Data_loader(paths,is_label)
    data_loader=DataLoader(data_loader, batch_size=8, num_workers=2)  
    data=[]
    for x in tqdm(data_loader):
        data.append(x)
    x=tc.cat(data,dim=0)
    del data
    if not is_label:
        ########################################################################
        TH=x.reshape(-1).numpy()
        index = -int(len(TH) * CFG.chopping_percentile)
        TH:int = np.partition(TH, index)[index]
        x[x>TH]=int(TH)
        ########################################################################
        TH=x.reshape(-1).numpy()
        index = -int(len(TH) * CFG.chopping_percentile)
        TH:int = np.partition(TH, -index)[-index]
        x[x<TH]=int(TH)
        ########################################################################
        x=(min_max_normalization(x.to(tc.float16)[None])[0]*255).to(tc.uint8)
    return x


#https://www.kaggle.com/code/kashiwaba/sennet-hoa-train-unet-simple-baseline
def dice_coef(y_pred:tc.Tensor,y_true:tc.Tensor, thr=0.5, dim=(-1,-2), epsilon=0.001):
    #y_pred=y_pred.sigmoid()
    y_true = y_true.to(tc.float32)
    y_pred = (y_pred>thr).to(tc.float32)
    inter = (y_true*y_pred).sum(dim=dim)
    den = y_true.sum(dim=dim) + y_pred.sum(dim=dim)
    dice = ((2*inter+epsilon)/(den+epsilon)).mean()
    return dice

class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        
        #comment out if your model contains a sigmoid or equivalent activation layer
        inputs = inputs.sigmoid()   
        
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        
        intersection = (inputs * targets).sum()                            
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)  
        
        return 1 - dice

class SurfaceDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SurfaceDiceLoss, self).__init__()

    def forward(self, pred, surface_true, volume_true, smooth=1, k_size=3):
        pred = pred.sigmoid()
        surface_pred = pred * surface_true
        volume_pred = pred * volume_true
      
        surface_pred = surface_pred.view(-1)
        surface_true = surface_true.view(-1)
        volume_pred = volume_pred.view(-1)
        volume_true = volume_true.view(-1)
        
        surface_intersection = (surface_pred * surface_true).sum()
        volume_intersection = (volume_pred * volume_true).sum()

        surface_dice = (2. * (surface_intersection + (0.35*volume_intersection)) + smooth) / (surface_pred.sum() + surface_true.sum() + (0.35*volume_pred.sum()) + (0.35*volume_true.sum()) + smooth)

        return 1 - surface_dice


class Kaggld_Dataset(Dataset):
    def __init__(self,x:list,y:list,arg=False):
        super(Dataset,self).__init__()
        self.x=x#list[(C,H,W),...]
        self.y=y#list[(C,H,W),...]
        self.image_size=CFG.image_size
        self.in_chans=CFG.in_chans
        self.arg=arg
        if arg:
            self.transform=CFG.train_aug
        else: 
            self.transform=CFG.valid_aug

    def __len__(self) -> int:
        return sum([y.shape[0]-self.in_chans for y in self.y])
    
    def __getitem__(self,index):
        i=0
        for x in self.x:
            if index>x.shape[0]-self.in_chans:
                index-=x.shape[0]-self.in_chans
                i+=1
            else:
                break
        x=self.x[i]
        y=self.y[i]
        
        print (f'x.shape[1] ={x.shape[1]}    x.shape[2]={x.shape[2]}')
        
        x_index= (x.shape[1]-self.image_size)//2 #np.random.randint(0,x.shape[1]-self.image_size)
        y_index= (x.shape[2]-self.image_size)//2 # np.random.randint(0,x.shape[2]-self.image_size)
        # i i+5 
        x=x[index:index+self.in_chans   ,   x_index:x_index+self.image_size,   y_index:y_index+self.image_size]
        # i+2
        y=y[index+self.in_chans//2   ,      x_index:x_index+self.image_size,   y_index:y_index+self.image_size]

        data = self.transform(image=x.numpy().transpose(1,2,0), mask=y.numpy())
        x = data['image']
        y = data['mask']>=127
        if self.arg:
            i=np.random.randint(4)
            x=x.rot90(i,dims=(1,2))
            y=y.rot90(i,dims=(0,1))
            for i in range(3):
                if np.random.randint(2):
                    x=x.flip(dims=(i,))
                    if i>=1:
                        y=y.flip(dims=(i-1,))
        return x,y#(uint8,uint8)

In [5]:
tc.backends.cudnn.enabled = True
tc.backends.cudnn.benchmark = True

In [6]:

# Mixture of Experts implementation inspired by [https://arxiv.org/pdf/2208.02813.pdf]

class Router(nn.Module):
    def __init__(self, CFG, weight=None):
        super().__init__()

        self.encoder = smp.Unet(
            encoder_name=CFG.router_backbone, 
            encoder_weights=weight,
            in_channels=CFG.in_chans,
            classes=CFG.target_size,
            activation=None,
        ).encoder
        self.classifier = nn.Sequential(
            nn.AdaptiveMaxPool2d(1),
            nn.Flatten(),
            nn.Linear(2048, len(CFG.backbones)),
        )

    def forward(self, x):
        z = self.encoder(x)[-1]
        z = self.classifier(z)
        return z

class CustomModel_MoE(nn.Module):
    def __init__(self, CFG, topk: int = 2, weight=None):
        super().__init__()
        assert 0 < topk <= len(CFG.backbones), f"topk should be in (0, {len(CFG.backbones)}]"
        self.topk = topk

        self.router = Router(CFG, weight)
        self.experts = nn.ModuleList([smp.Unet(
            encoder_name=backbone, 
            encoder_weights=weight,
            in_channels=CFG.in_chans,
            classes=CFG.target_size,
            activation=None,
        ) for backbone in CFG.backbones])
        self.register_buffer('n_experts', tc.tensor(len(CFG.backbones), dtype=tc.float32))

    def forward(self, image):
        router_logits = self.router(image)
        preds_list = [expert(image)[:, 0].sigmoid() for expert in self.experts]
        return preds_list, router_logits

    def predict(self, image):
        # Distribute inputs to top-k experts with router
        dist = nn.functional.softmax(self.router(image), dim=-1)

        batch_size, _num_classes = dist.size()
        top_k_values, top_k_indices = tc.topk(dist, self.topk, dim=1)
        batch_indices = tc.arange(batch_size).unsqueeze(1).expand(-1, self.topk)

        k_hot_weights = tc.zeros_like(dist)
        k_hot_outputs = tc.zeros_like(dist)
        k_hot_outputs[batch_indices, top_k_indices] = 1
        k_hot_weights[batch_indices, top_k_indices] = top_k_values / top_k_values.sum(dim=-1, keepdim=True)

        agg_pred = tc.zeros_like(image, dtype=tc.float32).squeeze(1)
        for i in range(len(self.experts)):
            idx = k_hot_outputs[:, i].bool()
            if idx.sum() > 0:
                agg_pred[idx] += self.experts[i](image[idx])[:, 0].sigmoid() * k_hot_weights[idx, i].unsqueeze(-1).unsqueeze(-1)
        
        return agg_pred

def build_model(weight="imagenet"):
    load_dotenv()

    print('model_name', CFG.model_name)
    print('router_backbone', CFG.router_backbone)
    print('expert_backbones', CFG.backbones)

    model = CustomModel_MoE(CFG, topk=1, weight=weight)

    return model.cuda()

In [7]:
class CustomModelLoad(nn.Module):
    def __init__(self, CFG, weight=None):
        super().__init__()
        self.CFG = CFG
        self.model = build_model()
        self.batch=CFG.batch
    
    def forward(self,x:tc.Tensor):
        #x.shape=(batch,c,h,w)
        x=x.to(tc.float32)
        x=norm_with_clip(x.reshape(-1,*x.shape[2:])).reshape(x.shape)
        
        if CFG.input_size!=CFG.image_size:
            x=nn.functional.interpolate(x,size=(CFG.input_size,CFG.input_size),mode='bilinear',align_corners=True)
        
        shape=x.shape
        x=[tc.rot90(x,k=i,dims=(-2,-1)) for i in range(4)]
        x=tc.cat(x,dim=0)
        with autocast():
            with tc.no_grad():
                x=[self.model.predict(x[i*self.batch:(i+1)*self.batch]) for i in range(x.shape[0]//self.batch+1)]
                # batch=64,64...48
                x=tc.cat(x,dim=0)
        # x=x.sigmoid() # sigmoid is done inside predict
        x=x.reshape(4,shape[0],*shape[2:])
        x=[tc.rot90(x[i],k=-i,dims=(-2,-1)) for i in range(4)]
        x=tc.stack(x,dim=0).mean(0)
        
        if CFG.input_size!=CFG.image_size:
            x=nn.functional.interpolate(x[None],size=(CFG.image_size,CFG.image_size),mode='bilinear',align_corners=True)[0]
        return x

In [8]:
# data_dir = '/kaggle/input/blood-vessel-segmentation'
# model_pth = "/kaggle/input/backboned-model/backboned_model.pt"
data_dir = '/root/data'
model_pth = './bin/_tmp_models/unet_MoE.pt'

is_submit = len(glob(f"{data_dir}/test/kidney_5/images/*.tif"))!=3
scan_folders = glob(f"{data_dir}/test/*/")
if not is_submit:
    scan_folders = [
        # f'{data_dir}/train/kidney_2/'
        f'{data_dir}/train/kidney_3_dense/'
    ]
is_dummy_run = False # not is_submit

model_path_i = 0 
class CFG_Load:
    model_name = 'Unet'
    # backbone = 'resnext50_32x4d'
    router_backbone = 'resnext50_32x4d'
    backbones = [
        'resnext50_32x4d',
        'resnext50_32x4d',
        'resnext50_32x4d',
        'resnext50_32x4d',
        # 'resnext101_32x8d',
        # 'se_resnext50_32x4d',
        # 'mit_b2'
    ]

    in_chans = 1
    image_size = 1024 
    input_size= 1024
    tile_size = image_size
    stride = tile_size // 4
    drop_egde_pixel= 0 
    
    target_size = 1
    chopping_percentile=1e-3
    valid_id = 1
    batch=16 
    th_percentile = 0.00143
    
    path_submition = 0 
    model_path=[model_pth]

def build_model_load(weight=None):
    load_dotenv()
    model = CustomModelLoad(CFG_Load, weight)
    return model.cuda()

In [9]:
model_load=build_model_load()
model_load.model.load_state_dict(tc.load(CFG_Load.model_path[model_path_i],"cpu"))
model_load.eval()

model_name Unet
router_backbone resnext50_32x4d
expert_backbones ['resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d', 'resnext50_32x4d']


CustomModelLoad(
  (model): CustomModel_MoE(
    (router): Router(
      (encoder): ResNetEncoder(
        (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): BatchNorm2d(256, eps=1e-05

In [10]:
def rle_encode(mask):
    pixel = mask.flatten()
    pixel = np.concatenate([[0], pixel, [0]])
    run = np.where(pixel[1:] != pixel[:-1])[0] + 1
    run[1::2] -= run[::2]
    rle = ' '.join(str(r) for r in run)
    if rle == '':
        rle = '1 0'
    return rle

def rle_encode_new(mask):
    pixel = mask.flatten()
    pixel = np.concatenate([[0], pixel, [0]])
    run = np.where(pixel[1:] != pixel[:-1])[0] + 1
    #run = np.diff(run)
    # check the length of run
    #if len(run) % 2 != 0:
        # append a zero to the end
        #run = np.append(run, 0)
    # perform the subtraction
    run[1::2] -= run[::2]
    rle = ' '.join(str(r) for r in run)
    if rle == '':
        rle = '1 0'
    return rle


def min_max_normalization(x:tc.Tensor)->tc.Tensor:
    """input.shape=(batch,f1,...)"""
    shape=x.shape
    if x.ndim>2:
        x=x.reshape(x.shape[0],-1)
    
    min_=x.min(dim=-1,keepdim=True)[0]
    max_=x.max(dim=-1,keepdim=True)[0]
    if min_.mean()==0 and max_.mean()==1:
        return x.reshape(shape)
    
    x=(x-min_)/(max_-min_+1e-9)
    return x.reshape(shape)

def norm_with_clip(x:tc.Tensor,smooth=1e-5):
    dim=list(range(1,x.ndim))
    mean=x.mean(dim=dim,keepdim=True)
    std=x.std(dim=dim,keepdim=True)
    x=(x-mean)/(std+smooth)
    x[x>5]=(x[x>5]-5)*1e-3 +5
    x[x<-3]=(x[x<-3]+3)*1e-3-3
    return x

class Data_loader(Dataset):
    def __init__(self,path,s="/images/"):
        self.paths=glob(path+f"{s}*.tif")
        self.paths.sort()
        self.bool=s=="/labels/"
    
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self,index):
        img=cv2.imread(self.paths[index],cv2.IMREAD_GRAYSCALE)
        img = to_1024_1024(img , image_size = CFG.image_size )
        
        img=tc.from_numpy(img.copy())
        if self.bool:
            img=img.to(tc.bool)
        else:
            img=img.to(tc.uint8)
        return img

def load_data(path,s):
    data_loader=Data_loader(path,s)
    data_loader=DataLoader(data_loader, batch_size=16, num_workers=2)
    data=[]
    for x in tqdm(data_loader):
        data.append(x)
    x=tc.cat(data,dim=0)
    ########################################################################
    TH=x.reshape(-1).numpy()
    index = -int(len(TH) * CFG.chopping_percentile)
    TH:int = np.partition(TH, index)[index]
    x[x>TH]=int(TH)
    ########################################################################
    TH=x.reshape(-1).numpy()
    index = -int(len(TH) * CFG.chopping_percentile)
    TH:int = np.partition(TH, -index)[-index]
    x[x<TH]=int(TH)
    ########################################################################
    #x=(min_max_normalization(x.to(tc.float16))*255).to(tc.uint8)
    return x

class Pipeline_Dataset(Dataset):
    def __init__(self,x,path):
        self.img_paths  = glob(path+"images/*")
        self.img_paths.sort()
        self.in_chan = CFG.in_chans
        z=tc.zeros(self.in_chan//2,*x.shape[1:],dtype=x.dtype)
        self.x=tc.cat((z,x,z),dim=0)
        
    def __len__(self):
        return self.x.shape[0]-self.in_chan+1
    
    def __getitem__(self, index):
        x  = self.x[index:index+self.in_chan]
        return x,index
    
    def get_mark(self,index):
        id=self.img_paths[index].split("/")[-3:]
        id.pop(1)
        id="_".join(id)
        return id[:-4]
    
    def get_marks(self):
        ids=[]
        for index in range(len(self)):
            ids.append(self.get_mark(index))
        return ids

def add_edge(x:tc.Tensor,edge:int):
    #x=(C,H,W)
    #output=(C,H+2*edge,W+2*edge)
    mean_=int(x.to(tc.float32).mean())
    x=tc.cat([x,tc.ones([x.shape[0],edge,x.shape[2]],dtype=x.dtype,device=x.device)*mean_],dim=1)
    x=tc.cat([x,tc.ones([x.shape[0],x.shape[1],edge],dtype=x.dtype,device=x.device)*mean_],dim=2)
    x=tc.cat([tc.ones([x.shape[0],edge,x.shape[2]],dtype=x.dtype,device=x.device)*mean_,x],dim=1)
    x=tc.cat([tc.ones([x.shape[0],x.shape[1],edge],dtype=x.dtype,device=x.device)*mean_,x],dim=2)
    return x

In [16]:
def get_output(paths: str, dummy_output: bool, debug: bool = False):
    #paths=["/root/data/train/kidney_2"]
    outputs=[[],[]]
    
    for path in paths:
        x=load_data(path,"images/")
        labels=tc.zeros_like(x,dtype=tc.uint8)
        mark=Pipeline_Dataset(x,path).get_marks()
        if dummy_output:
            outputs[0].append(labels)
            outputs[1].append(mark)
            continue
        
        do_axes = [1]
        for axis in do_axes:
            debug_count=0
            if axis==0:
                x_=x
                labels_=labels
            elif axis==1:
                x_=x.permute(1,2,0)
                labels_=labels.permute(1,2,0)
            elif axis==2:
                x_=x.permute(2,0,1)
                labels_=labels.permute(2,0,1)
            if x.shape[0]==3 and axis!=0:
                break
            dataset=Pipeline_Dataset(x_,path)
            dataloader=DataLoader(dataset,batch_size=1,shuffle=False,num_workers=1)
            shape=dataset.x.shape[-2:]
            x1_list = np.arange(0, shape[0]+CFG_Load.tile_size-CFG_Load.tile_size+1, CFG_Load.stride)
            y1_list = np.arange(0, shape[1]+CFG_Load.tile_size-CFG_Load.tile_size+1, CFG_Load.stride)
            for img,index in tqdm(dataloader):
                #img=(1,C,H,W)
                img=img.to("cuda:0")
                img=add_edge(img[0],CFG_Load.tile_size//2)[None]

                mask_pred = tc.zeros_like(img[:,0],dtype=tc.float32,device=img.device)
                mask_count = tc.zeros_like(img[:,0],dtype=tc.float32,device=img.device)

                indexs=[]
                chip=[]
                for y1 in y1_list:
                    for x1 in x1_list:
                        x2 = x1 + CFG_Load.tile_size
                        y2 = y1 + CFG_Load.tile_size
                        indexs.append([x1+CFG_Load.drop_egde_pixel,x2-CFG_Load.drop_egde_pixel,
                                       y1+CFG_Load.drop_egde_pixel,y2-CFG_Load.drop_egde_pixel])
                        chip.append(img[...,x1:x2,y1:y2])

                y_preds = model_load.forward(tc.cat(chip)).to(device=0)

                if CFG_Load.drop_egde_pixel:
                    y_preds=y_preds[...,CFG_Load.drop_egde_pixel:-CFG_Load.drop_egde_pixel,
                                        CFG_Load.drop_egde_pixel:-CFG_Load.drop_egde_pixel]
                for i,(x1,x2,y1,y2) in enumerate(indexs):
                    mask_pred[...,x1:x2, y1:y2] += y_preds[i]
                    mask_count[...,x1:x2, y1:y2] += 1

                mask_pred /= mask_count

                #Rrecover
                mask_pred=mask_pred[...,CFG_Load.tile_size//2:-CFG_Load.tile_size//2,CFG_Load.tile_size//2:-CFG_Load.tile_size//2]
                
                labels_[index]+=(mask_pred[0]*255 /len(do_axes) ).to(tc.uint8).cpu()
                if debug:
                    debug_count+=1
                    plt.subplot(121)
                    plt.imshow(img[0,CFG_Load.in_chans//2].cpu().detach().numpy())
                    plt.subplot(122)
                    plt.imshow(mask_pred[0].cpu().detach().numpy())
                    plt.show()
                    if debug_count>3:
                        break
        outputs[0].append(labels)
        outputs[1].append(mark)
    return outputs

In [12]:
outputs, ids_list = get_output(scan_folders, is_dummy_run)

100%|██████████| 501/501 [32:08<00:00,  3.85s/it]


In [71]:
tc.save(outputs, "tmp_k3dense_pred.pt")

In [15]:
#img=cv2.imread("/root/data/train/kidney_2/images/0000.tif", cv2.IMREAD_GRAYSCALE)
#img_kidney_5=cv2.

submission_df=[]
for output, ids in zip(outputs, ids_list):
    output = output > 75
    for index in range(len(ids)):
        id=ids[index]
        # i=0
        # for x in output:
        #     if index>=len(x):
        #         index-=len(x)
        #         i+=1
        #     else:
        #         break
        mask_pred=(output[index,:,:]).numpy()
        #mask_pred2 = to_original(mask_pred, img, image_size=1024)
        #mask_pred = mask_pred2.copy()

        rle = rle_encode(mask_pred)

        submission_df.append(
            pd.DataFrame(data={
                'id' : id,
                'rle' : rle,
            }, index=[0])
        )

submission_df = pd.concat(submission_df)
submission_df.to_csv('submission.csv', index=False)
submission_df.head(6)

,id,rle
0,kidney_3_dense_0496,352510 2 354019 4 355529 4 357040 2 367648 3 3...
0,kidney_3_dense_0497,352509 3 354018 4 355529 3 357040 1 363121 1 3...
0,kidney_3_dense_0498,278382 1 279891 2 281401 1 350999 1 352508 3 3...
0,kidney_3_dense_0499,260258 3 261748 3 261752 2 261767 5 263257 8 2...
0,kidney_3_dense_0500,349488 1 350997 3 352507 4 354017 3 355528 2 3...
0,kidney_3_dense_0501,349487 2 350997 3 352507 3 354018 2 355571 4 3...


In [18]:
# Bullshit (dont submit)

import sys
sys.path.append('..')
import util
import numpy as np
import pandas as pd
import torch as tc

In [20]:
util.Display.view([
    util.Display(outputs[0].unsqueeze(0)>85),
    util.Display(train.labels[0])
])

interactive(children=(IntSlider(value=0, description='i', max=500), Output()), _dom_classes=('widget-interact'…

In [19]:
patch_size = 1,256,256
train = util.data.SenNet(
    patch_size,
    guarantee_vessel=0,
    samples=[
        # "/train/kidney_1_dense",
        # "/train/kidney_2",
        "/train/kidney_3_dense"
    ]
)

Loading /train/kidney_3_dense/images from cache
Loading /train/kidney_3_dense/labels from cache


In [4]:
k2, k2lbl = train.scans[0], train.labels[0]

In [5]:
kidney_id = "kidney_2"
submit = pd.read_csv('submission.csv')
idx = submit['id'].str.startswith(kidney_id)
submit = submit[idx]
submit.reset_index(drop=True, inplace=True)

def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

# slices = []
# for row in submission_df.itertuples():
#     slices.append()
pmask2 = tc.stack(
    [tc.tensor(rle_decode(row.rle, util.data.KIDNEY_2_SHAPE))
    for row in submit.itertuples()]
).unsqueeze(0).bool()


In [10]:
pmask2.shape, k2lbl.shape

(torch.Size([1, 2217, 1041, 1511]), torch.Size([1, 2217, 1041, 1511]))

In [37]:
submit

,id,rle
0,kidney_2_0000,1 0
1,kidney_2_0001,1 0
2,kidney_2_0002,1 0
3,kidney_2_0003,1 0
4,kidney_2_0004,1 0
...,...,...
2212,kidney_2_2212,552197 2 617261 1 618741 3 618772 1 620252 2 6...
2213,kidney_2_2213,552198 1 573453 1 618741 2 618772 1 620215 1 6...
2214,kidney_2_2214,549216 1 550687 1 573453 1 608262 1 618772 1 6...
2215,kidney_2_2215,550687 1 573453 2 574964 1 608262 1 609773 1 6...


In [8]:
util.Display.view([
    util.Display(pmask2[:,:]),
    util.Display(k2lbl[:,:])
])

interactive(children=(IntSlider(value=0, description='i', max=2216), Output()), _dom_classes=('widget-interact…

In [9]:
1040 - 1024

16

In [76]:
#img=cv2.imread("/root/data/train/kidney_2/images/0000.tif", cv2.IMREAD_GRAYSCALE)
#img_kidney_5=cv2.
is_dummy_run = True
submission_df=[]
if not is_dummy_run:
    for output, ids in zip(outputs, ids_list):
        output = output > 75
        for index in range(len(ids)):
            id=ids[index]
            # i=0
            # for x in output:
            #     if index>=len(x):
            #         index-=len(x)
            #         i+=1
            #     else:
            #         break
            mask_pred=(output[index,:,:]).numpy()
            #mask_pred2 = to_original(mask_pred, img, image_size=1024)
            #mask_pred = mask_pred2.copy()

            rle = rle_encode(mask_pred)

            submission_df.append(
                pd.DataFrame(data={
                    'id' : id,
                    'rle' : rle,
                }, index=[0])
            )
else:
    submission_df.append(pd.DataFrame())
submission_df = pd.concat(submission_df)
submission_df.to_csv('tst_submission.csv', index=False)
submission_df.head(6)

""
